In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
%cd '/content/gdrive/My Drive/IDLSProject-main-2'

/content/gdrive/My Drive/IDLSProject-main-2


In [3]:
import pandas as pd
import numpy as np
import os
import pickle
import torch
import torch.utils.data
from sklearn.model_selection import train_test_split

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_qrnn.pkl'), "rb") as f:
    word_dict = pickle.load(f)

train = pd.read_csv(os.path.join(data_dir, 'train_qrnn.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_qrnn.csv'), header=None, names=None)

test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape


# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10000, 1])


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10
batch_size = 50
learning_rate = 0.001
seq_len = 500
dropout = 0.5
filter_size = 100
vocab_size = 5000
embed_dims = 32
hidden_size = 100
kernel_size = [3,4,5]

In [5]:
data_dir = './cnn-lstm-imdb-hyperband-trails/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./cnn-lstm-imdb-hyperband-trails/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [6]:
import torch
import torch.nn as nn
import numpy as np
class Combine(nn.Module):
    def __init__(self,trial,vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len,hidden_size):
        super(Combine, self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        hidden_size = trial.suggest_int("hidden_size",32,256)
        embed_size = trial.suggest_int("embed_size",16,128)
        n_filters = trial.suggest_int("n_filters",50,200)
        self.embedding = torch.nn.Embedding(vocab_size,embed_size)
        self.conv1 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[0],embed_size])
        self.conv2 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[1],embed_size])
        self.conv3 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[2],embed_size])
        self.mp1 = torch.nn.MaxPool1d(seq_len+1-kernel_size[0])
        self.mp2 = torch.nn.MaxPool1d(seq_len+1-kernel_size[1])
        self.mp3 = torch.nn.MaxPool1d(seq_len+1-kernel_size[2])
        self.dropout = torch.nn.Dropout(dropout)
        #self.cnn = CNN(vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len)
        self.lstm = nn.LSTM(
            input_size=3*filter_size, 
            hidden_size=hidden_size, 
            num_layers=1,
            batch_first=True)
        self.dense = nn.Linear(hidden_size,1)
        self.sig = nn.Sigmoid()
        self.word_dict = None


    def forward(self, x):
        
        embed_input = self.embedding(x)
        embed_input.unsqueeze_(1)
        x1 = torch.tanh(self.dropout(self.conv1(embed_input))).squeeze(3)
        # print("x1")
        # print(x1.shape)
        x2 = torch.tanh(self.dropout(self.conv2(embed_input))).squeeze(3)
        x3 = torch.tanh(self.dropout(self.conv3(embed_input))).squeeze(3)
        f1 = self.mp1(x1).squeeze(2)
        # print("f1")
        # print(f1.shape)
        f2 = self.mp2(x2).squeeze(2)
        f3 = self.mp3(x3).squeeze(2)
        c_out = torch.cat([f1,f2,f3],dim=1)
        r_in = c_out.view(batch_size,1, -1)
        lstm_out, _ = self.lstm(r_in)
        out = self.dense(lstm_out[:, -1, :])
        return self.sig(out)

In [15]:
import time
import numpy as np 
def objective(trial):
  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
  #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = Combine(trial,vocab_size, embed_dims, filter_size, kernel_size, dropout, seq_len,hidden_size).to(device).to(device)
  trial.set_user_attr(key="best_model", value=model)
  optimizer = optim.Adam(model.parameters())
  loss_fn = torch.nn.BCELoss()
  train_loss_epoch = []
  train_acc_epoch = []
  val_loss_epoch = []
  val_accuracy_epoch = []
  time_train = []
  val_acc = 0
  for epoch in range(epochs):
    start = time.time()
    model.train()
    total_loss = 0
    train_acc = 0
    total = 0
    correct = 0
    for batch in train_dl:         
      batch_X, batch_y = batch
      batch_X = batch_X.to(device)
      batch_y = batch_y.to(device)
      optimizer.zero_grad()
      prediction = model(batch_X)
      loss = loss_fn(prediction, batch_y)
      loss.backward()
      optimizer.step()
      result = np.round(prediction.detach().cpu())
      total_loss += loss.data.item()
      total += batch_y.size(0)
      correct += (result == batch_y.cpu()).sum().item()
      train_acc = correct/total
    train_loss_epoch.append(np.round(total_loss / len(train_dl), 3))
    train_acc_epoch.append(np.round(train_acc*100,3))
    print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
    with torch.no_grad():
      model.eval()
      correct = 0
      total = 0
      val_loss = []
      for inputs, labels in val_dl:
        inputs_val, labels_val = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        prediction = model(inputs_val)
        loss = loss_fn(prediction, labels_val)
        val_loss.append(loss.item())
        result = np.round(prediction.cpu())
        total += labels_val.size(0)
        correct += (result == labels_val.cpu()).sum().item()
      val_accuracy_epoch.append(np.round((correct/total)*100, 3))
      val_loss_epoch.append(np.round(np.mean(val_loss),3))
      end = time.time() - start
      print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
      time_train.append(np.round(end,3))
      val_acc = np.round((correct/total)*100, 3)
    write_to_csv(trial.number,epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_accuracy_epoch, time_train)
    return val_acc


def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    acc = correct/total
    print("Accuracy:", (correct/total)*100)

In [8]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [9]:
!pip install optuna

In [10]:
import torch.optim as optim
epochs = 10

In [11]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=5,callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2022-05-14 19:19:41,946] A new study created in memory with name: no-name-d07ec7e0-5ea4-48c7-ab91-69cfc99a2338


Epoch: 0, BCELoss: 0.582429970651865


[I 2022-05-14 19:19:55,928] Trial 0 finished with value: 79.12 and parameters: {'optimizer': 'Adam', 'lr': 9.296672919567677e-05, 'dropout': 0.17128277945653755, 'hidden_size': 181, 'embed_size': 42, 'n_filters': 188}. Best is trial 0 with value: 79.12.


Val Loss: 0.447 	Val Acc: 0.791
Epoch: 0, BCELoss: 0.5636818692584833


[I 2022-05-14 19:20:04,282] Trial 1 finished with value: 81.89 and parameters: {'optimizer': 'SGD', 'lr': 0.17408070875658735, 'dropout': 0.18453676364573948, 'hidden_size': 139, 'embed_size': 50, 'n_filters': 92}. Best is trial 1 with value: 81.89.


...

Val Loss: 0.410 	Val Acc: 0.819
Epoch: 0, BCELoss: 0.5562615537643433


[I 2022-05-14 19:20:22,839] Trial 3 finished with value: 83.36 and parameters: {'optimizer': 'RMSprop', 'lr': 0.12360115688111847, 'dropout': 0.19480660133684466, 'hidden_size': 69, 'embed_size': 95, 'n_filters': 51}. Best is trial 3 with value: 83.36.


Val Loss: 0.387 	Val Acc: 0.834
Epoch: 0, BCELoss: 0.6775266034404437


[I 2022-05-14 19:20:32,206] Trial 4 finished with value: 76.7 and parameters: {'optimizer': 'SGD', 'lr': 3.984785303484685e-05, 'dropout': 0.5476721857137145, 'hidden_size': 192, 'embed_size': 88, 'n_filters': 91}. Best is trial 3 with value: 83.36.


Val Loss: 0.513 	Val Acc: 0.767
Study statistics: 
  Number of finished trials:  5
  Number of complete trials:  5
Best trial:
  Value:  83.36
  Params: 
    optimizer: RMSprop
    lr: 0.12360115688111847
    dropout: 0.19480660133684466
    hidden_size: 69
    embed_size: 95
    n_filters: 51


In [12]:
torch.save(best_model.state_dict(),"./cnn-lstm-imdb-hyperband.pth")

In [16]:
test(best_model, test_dl, epochs)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:762: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:926.)
  self.dropout, self.training, self.bidirectional, self.batch_first)


Accuracy: 83.95
